In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import os
import numpy as np
from sklearn.metrics import classification_report
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
from tqdm import tqdm

drive_results_dir = "./results/t5-flan"
os.makedirs(drive_results_dir, exist_ok=True)

model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

dataset = load_dataset("rotten_tomatoes")
test_data = dataset["test"]

def classify_sentiment(example, model, tokenizer):
    prompt = f"Classify the sentiment: {example['text']}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=128)
    outputs = model.generate(**inputs)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction.strip()

predictions = []
true_labels = []
example_queries = []
label_map = {"negative": 0, "positive": 1}

for idx, example in enumerate(tqdm(test_data)):
    true_label = example["label"]
    predicted_label = classify_sentiment(example, model, tokenizer)

    if predicted_label in label_map:
        predictions.append(label_map[predicted_label])
    else:
        print(f"Unexpected label: {predicted_label}, skipping example.")
        continue

    true_labels.append(true_label)

    if idx < 5: #change for more predicion examples
        example_queries.append({
            "text": example["text"],
            "true_label": "positive" if true_label == 1 else "negative",
            "predicted_label": predicted_label
        })

print("Generating detailed evaluation report...")
report = classification_report(true_labels, predictions, target_names=["negative", "positive"], digits=4)

classification_report_file = os.path.join(drive_results_dir, "classification_report.txt")
with open(classification_report_file, "w") as f:
    f.write(report)

print(f"Classification report saved at: {classification_report_file}")

print("Classification Report:\n", report)

print("\nSample predictions:")
for example in example_queries:
    print(f"Text: {example['text']}")
    print(f"True Label: {example['true_label']}, Predicted Label: {example['predicted_label']}")
    print("-" * 50)


  3%|▎         | 30/1066 [00:02<01:32, 11.15it/s]

Unexpected label: warm, skipping example.


 71%|███████   | 758/1066 [01:18<00:33,  9.14it/s]

Unexpected label: less, skipping example.


100%|██████████| 1066/1066 [01:47<00:00,  9.90it/s]

Generating detailed evaluation report...
Classification report saved at: ./results/t5-flan/classification_report.txt
Classification Report:
               precision    recall  f1-score   support

    negative     0.8000    0.8571    0.8276       532
    positive     0.8462    0.7857    0.8148       532

    accuracy                         0.8214      1064
   macro avg     0.8231    0.8214    0.8212      1064
weighted avg     0.8231    0.8214    0.8212      1064


Sample predictions:
Text: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
True Label: positive, Predicted Label: positive
--------------------------------------------------
Text: consistently clever and suspenseful .
True Label: positive, Predicted Label: positive
--------------------------------------------------
Text: it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political a